In [ ]:
# Install all dependencies
# %pip install weblinx[all]

In [ ]:
from pathlib import Path
import weblinx as wl


wl_dir = Path("/ai_dev/cache/hub/datasets--McGill-NLP--WebLINX-full/snapshots/9ac27363e3e340a06fb9df7785a6657be48207a7")
base_dir = wl_dir / "demonstrations"
# split_path = wl_dir / "splits.json"

# Load the name of the demonstrations in the training split
#demo_names = wl.utils.load_demo_names_in_split(split_path, split='train')
# You can use: train, valid, test_iid, test_vis, test_cat, test_geo, test_web
# Or you can specify the demo_names yourself, such as the ones we just fetched
demo_names = ['aaabtsd', 'aajfwoq', 'aovxytv']  # 3 random demo from valid

# Load the demonstrations
demos = [wl.Demonstration(name, base_dir=base_dir) for name in demo_names]

# Select a demo to work with
demo = demos[0]

# Load the Replay object, which contains the turns of the demonstration
replay = wl.Replay.from_demonstration(demo)

# Filter the turns to keep only the ones that are relevant for the task
turns = replay.filter_by_intents(
    "click", "textInput", "load", "say", "submit"
)

# Only keep the turns that have a good screenshot (i.e., the screenshot is not empty)
turns = wl.filter_turns(
    turns, lambda t: t.has_screenshot() and t.get_screenshot_status() == "good"
)

# Remove chat turns where the speaker is not the navigator (e.g. if you want to train a model to predict the next action)
turns = wl.filter_turns(
    turns,
    lambda turn: not (
        turn.type == "chat" and turn.get("speaker") != "navigator"
    ),
)

In [ ]:
turns = replay.filter_turns(lambda x: True)

In [ ]:
len(replay.data_dict)

In [ ]:
for element in replay.data_dict:
    # print(element.keys())
    e_type = element["type"]
    print(e_type)
    if e_type == "chat":
        print(element["speaker"])
        print(element["utterance"])
    elif e_type == "browser":
        print(element["state"].keys())
        print(element["action"].keys())
        intent = element["action"]["intent"]
        arguments = element["action"]["arguments"]
        print(intent)
        print(arguments.keys())
        print(arguments["properties"])
    else:
        print("UNKOWN ELEMENT TYPE")
    print("#" * 80)
    

In [ ]:
turns = replay.filter_if_screenshot()
turns

In [ ]:
for turn in turns:
    print(turn.intent)
    print(turn.args)

In [ ]:
t = turns[0]

In [ ]:
t.type

In [ ]:
t.args

In [ ]:
turns[0]

In [ ]:
# Optional: We can even get the path to the screenshot and open with Pillow
# Install Pillow with: pip install Pillow
from PIL import Image
Image.open(turns[0].get_screenshot_path())

In [ ]:
turn.get_screenshot_path()

In [ ]:
from datasets import load_dataset
from huggingface_hub import snapshot_download

# Load the validation split
valid = load_dataset("McGill-NLP/weblinx", split="validation")

# Download the input templates and use the LLaMA one
snapshot_download(
    "McGill-NLP/WebLINX", repo_type="dataset", allow_patterns="templates/*", local_dir="."
)
with open('templates/llama.txt') as f:
    template = f.read()

# To get the input text, simply pass a turn from the valid split to the template
turn = valid[0]
turn_text = template.format(**turn)